In [1]:
from backtesting.test import EURUSD, SMA

data = EURUSD.copy()
data

C:\Users\LimJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\LimJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\LimJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\test\__init__.py

,Open,High,Low,Close,Volume
2017-04-19 09:00:00,1.07160,1.07220,1.07083,1.07219,1413
2017-04-19 10:00:00,1.07214,1.07296,1.07214,1.07260,1241
2017-04-19 11:00:00,1.07256,1.07299,1.07170,1.07192,1025
2017-04-19 12:00:00,1.07195,1.07280,1.07195,1.07202,1460
2017-04-19 13:00:00,1.07200,1.07230,1.07045,1.07050,1554
...,...,...,...,...,...
2018-02-07 11:00:00,1.23390,1.23548,1.23386,1.23501,2203
2018-02-07 12:00:00,1.23501,1.23508,1.23342,1.23422,2325
2018-02-07 13:00:00,1.23422,1.23459,1.23338,1.23372,2824
2018-02-07 14:00:00,1.23374,1.23452,1.23238,1.23426,4065


In [2]:
def BBANDS(data, n_lookback, n_std):
    """Bollinger bands indicator"""
    hlc3 = (data.High + data.Low + data.Close) / 3
    mean, std = hlc3.rolling(n_lookback).mean(), hlc3.rolling(n_lookback).std()
    upper = mean + n_std*std
    lower = mean - n_std*std
    return upper, lower


close = data.Close.values
sma10 = SMA(data.Close, 10)
sma20 = SMA(data.Close, 20)
sma50 = SMA(data.Close, 50)
sma100 = SMA(data.Close, 100)
upper, lower = BBANDS(data, 20, 2)

# Design matrix / independent features:

# Price-derived features
data['X_SMA10'] = (close - sma10) / close
data['X_SMA20'] = (close - sma20) / close
data['X_SMA50'] = (close - sma50) / close
data['X_SMA100'] = (close - sma100) / close

data['X_DELTA_SMA10'] = (sma10 - sma20) / close
data['X_DELTA_SMA20'] = (sma20 - sma50) / close
data['X_DELTA_SMA50'] = (sma50 - sma100) / close

# Indicator features
data['X_MOM'] = data.Close.pct_change(periods=2)
data['X_BB_upper'] = (upper - close) / close
data['X_BB_lower'] = (lower - close) / close
data['X_BB_width'] = (upper - lower) / close
data['X_Sentiment'] = ~data.index.to_series().between('2017-09-27', '2017-12-14')

# Some datetime features for good measure
data['X_day'] = data.index.dayofweek
data['X_hour'] = data.index.hour

data = data.dropna().astype(float)

In [3]:
import numpy as np

def create_sequences(data, sequence_length=30, prediction_steps=5):
    X, y = [], []
    data = data.dropna().reset_index(drop=True)
    for i in range(sequence_length, len(data) - prediction_steps + 1):
        X.append(data[['Close']].iloc[i-sequence_length:i].values)
        y.append(data['Close'].pct_change(prediction_steps).iloc[i + prediction_steps - 1])
    return np.array(X), np.array(y)


def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model



In [7]:
%%time

import pandas as pd

from backtesting import Backtest, Strategy

class LSTMStrategy(Strategy):
    n_train = 300

    def init(self):
        print("Initializing strategy...")
        # Ensure data is DataFrame and add 'returns' column
        if isinstance(self.data.df, pd.DataFrame):
            self.data.df['returns'] = self.data.df['Close'].pct_change()

            # Prepare data for LSTM
            print("Creating sequences...")
            self.X, self.y = create_sequences(self.data.df, 30, 5)
            self.X_train, self.y_train = self.X[:self.n_train], self.y[:self.n_train]

            # Build and train the model
            print("Building and training the model...")
            self.model = build_model((30, 1))
            self.model.fit(self.X_train, self.y_train, epochs=5, batch_size=64, verbose=1)
            print("Model training completed.")

    def next(self):
        print(f"Processing next step for data at index {len(self.data)}...")
        if len(self.data) < self.n_train + 30:
            print("Not enough data to predict. Skipping...")
            return

        # Get the latest data sequence for prediction
        X_last = self.data.df['Close'][-30:].values.reshape(1, 30, 1)
        prediction = self.model.predict(X_last)[0][0]
        print(f"Prediction: {prediction}")

        # Basic trading logic based on the prediction
        if prediction > 0:
            print("Predicted an upward movement. Placing a buy order.")
            self.buy()
        elif prediction < 0:
            print("Predicted a downward movement. Placing a sell order.")
            self.sell()



bt = Backtest(data, LSTMStrategy, commission=.0002, margin=.05, exclusive_orders=True)
results = bt.run()


Initializing strategy...
Creating sequences...
Building and training the model...
Epoch 1/5


C:\Users\LimJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0046
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0027
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0018
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0011
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 6.0360e-04
Model training completed.
Processing next step for data at index 2...
Not enough data to predict. Skipping...
Processing next step for data at index 3...
Not enough data to predict. Skipping...
Processing next step for data at index 4...
Not enough data to predict. Skipping...
Processing next step for data at index 5...
Not enough data to predict. Skipping...
Processing next step for data at index 6...
Not enough data to predict. Skipping...
Processing next step for data at index 7...
Not enough data to predict. Skipping...
Processing next step for data at index 8...
Not enough data to predict. Skipping...
Processing next step for data at index 9...
Not enough data to predict. Skipp

In [9]:
print(results)

Start                     2017-04-25 12:00:00
End                       2018-02-07 15:00:00
Duration                    288 days 03:00:00
Exposure Time [%]                   54.295042
Equity Final [$]                     0.057564
Equity Peak [$]                       10000.0
Return [%]                         -99.999424
Buy & Hold Return [%]               12.869869
Return (Ann.) [%]                  -99.999571
Volatility (Ann.) [%]                0.000751
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.999425
Avg. Drawdown [%]                  -99.999425
Max. Drawdown Duration      268 days 10:00:00
Avg. Drawdown Duration      268 days 10:00:00
# Trades                                 2658
Win Rate [%]                        34.875847
Best Trade [%]                       1.009808
Worst Trade [%]                     -0.695827
Avg. Trade [%]                    

In [8]:
bt.plot()

C:\Users\LimJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\LimJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\LimJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\LimJ\AppData\Local\Packages\PythonSof

GridPlot(id='p1305', ...)